# Soil Grids Collection for the European Alps
-------------
### Data Description
SoilGrids is a system for global digital soil mapping that uses state-of-the-art machine learning methods to map the spatial distribution of soil properties across the globe. SoilGrids prediction models are fitted using over 230,000 soil profile observations from the WoSIS database and a series of environmental covariates. Covariates were selected from a pool of over 400 environmental layers from Earth observation derived products and other environmental information including climate, land cover and terrain morphology. The outputs of SoilGrids are global soil property maps at six standard depth intervals (according to the GlobalSoilMap IUSS working group and its specifications) at a spatial resolution of 250 meters. The SoilGrids datasets for the European Alps was used in this example. 

### Details of example
**_Goal_:** _Create a Collection with specified scale and offset values_

In this example, we show a case where it is deemed necessary to include a scale and offset value as part of the dataset metadata. Since Raster2STAC reads these information (scale, offset, nodata values) directly for the raster data metadata, we showcase how to write/ change these values to the accurate versions that suits respective project needs. 




In [1]:
# import libraries
import os
import pathlib
import rioxarray
import xarray as xr
import sys
import json

### Load datasets from cloud storage -- soil grid bulk density

In [2]:
!wget https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl1_20200301000000.tif
!wget https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl2_20200301000000.tif
!wget https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl3_20200301000000.tif
!wget https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl4_20200301000000.tif
!wget https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl5_20200301000000.tif
!wget https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl6_20200301000000.tif

--2024-05-13 14:18:27--  https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl1_20200301000000.tif
Resolving eurac-eo.s3-eu-west-1.amazonaws.com (eurac-eo.s3-eu-west-1.amazonaws.com)... 3.5.65.171, 52.218.57.144, 52.218.98.160, ...
Connecting to eurac-eo.s3-eu-west-1.amazonaws.com (eurac-eo.s3-eu-west-1.amazonaws.com)|3.5.65.171|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14644761 (14M) [binary/octet-stream]
Saving to: ‘bd_sl1_20200301000000.tif’

bd_sl1_202003010000 100%[===================>]  13.97M  32.1MB/s    in 0.4s    

2024-05-13 14:18:28 (32.1 MB/s) - ‘bd_sl1_20200301000000.tif’ saved [14644761/14644761]

--2024-05-13 14:18:28--  https://eurac-eo.s3-eu-west-1.amazonaws.com/SOILGRIDS/bd_sl2_20200301000000.tif
Resolving eurac-eo.s3-eu-west-1.amazonaws.com (eurac-eo.s3-eu-west-1.amazonaws.com)... 52.218.57.144, 52.218.98.160, 52.92.17.82, ...
Connecting to eurac-eo.s3-eu-west-1.amazonaws.com (eurac-eo.s3-eu-west-1.amazonaws.com)|52.218.57.144|:

### Utility function
Function to write/rewrite scale and offset values to raster dataset

In [3]:

def add_scale_and_offset(
    data_path: str, scale_factor: float = 1.0, offset: float = 0.0, tiff: bool = True
):
    """write and save scale and offset values to datasets

    Args:
        data_path (str): path to the dataset
        scale_factor (float, optional): integer value of the scale_factor. Defaults to 1.0.
        offset (float, optional): integer value of the offset. Defaults to 0.0.
        tiff (bool, optional): Boolean if the dataset is in a TIFF format or not. Defaults to True.
    """
    if tiff:
        rio_ds = rioxarray.open_rasterio(data_path)
        rio_ds.attrs["scale_factor"] = scale_factor
        rio_ds.attrs["offset"] = offset

        # save the scale and offset to data
        rio_ds.rio.to_raster(data_path)
    else:
        xds = xr.open_dataset(data_path)
        xds.attrs["scale_factor"] = scale_factor
        xds.attrs["offset"] = offset
        
        # save scale and offset to data
        xds.to_netcdf(data_path)

### Add scale and offset value to datasets

In [4]:
soil_bulk_density_tiffs = sorted(pathlib.Path("./").glob("bd*.tif"))

for dpath in soil_bulk_density_tiffs:
    add_scale_and_offset(dpath, scale_factor=0.01, offset=0)

Check the dataset metadata

In [6]:
rio_ds = rioxarray.open_rasterio("bd_sl1_20200301000000.tif", chunks={})
rio_ds.attrs

{'AREA_OR_POINT': 'Area',
 'Code_version': 'v2.0.0',
 'Covariates': 'depth,clm_wcl_s07rad,clm_wcl_srdyrsum,clm_wcl_s08rad,luc_lds_mng,mor_mrg_vdp,clm_wcl_s04rad,clm_wcl_bio16,luc_gfc_trely10,clm_wcl_bio07,mor_env_demm,mor_mrg_twi,clm_esa_s09avg,clm_wcl_srdyrstd,luc_esa_l170,clm_mod_lstd01std,clm_esa_s05avg,clm_wcl_p05tot,mor_mrg_vbf,clm_wcl_p04tot,clm_wcl_bio18,clm_wcl_p10tot,clm_wcl_p09tot,clm_mod_lstd03std,veg_mod_evirng,veg_mod_eviyravg,clm_wcl_bio19,clm_mod_lstd09std,clm_wcl_p12tot,clm_wcl_bio08,clm_wcl_bio17,clm_mod_lstd11std,clm_mod_lstd05std,clm_mod_lstd02std,clm_esa_s08avg,clm_wcl_bio15,clm_mod_lstd10std,veg_mod_nppy15,clm_mod_lstd04std,clm_mod_lstd12std',
 'Litter_layers': 'FALSE',
 'long_name': 'bd_sl1',
 'Model': 'Quantile Regression Forests',
 'Model_type': 'ranger',
 'Mtry': 12,
 'Number_trees': 200,
 'offset': 0,
 'Outputs_abstract': 'no WISE, March 2020 snapshot, NZ and Siberia gaps',
 'Outputs_version': 'RUN03',
 'WoSIS_version': 'Data stream 7',
 '_FillValue': -32768,


### Generate STAC JSON using Raster2STAC

In [7]:
to_be_merged = [] 

for i, m in enumerate(soil_bulk_density_tiffs):
    file_tile = (str(m).split("/")[-1][:6])
    data = rioxarray.open_rasterio(m, chunks={})
    data = data.drop_vars("band").squeeze("band")
    data = data.to_dataset(name=f"{file_tile}")
    data = data.expand_dims(dim={"time": ["2024-03-01"]}, axis=0)

    to_be_merged.append(data)

bulk_density = xr.combine_by_coords(to_be_merged,combine_attrs="drop_conflicts")

bulk_density

<xarray.Dataset> Size: 244MB
Dimensions:      (time: 1, x: 6604, y: 3080)
Coordinates:
  * time         (time) object 8B '2024-03-01'
  * x            (x) float64 53kB 3.126 3.128 3.13 3.133 ... 16.88 16.88 16.88
  * y            (y) float64 25kB 50.03 50.03 50.03 50.02 ... 43.62 43.62 43.62
    spatial_ref  int64 8B 0
Data variables:
    bd_sl1       (time, y, x) int16 41MB dask.array<chunksize=(1, 3080, 6604), meta=np.ndarray>
    bd_sl2       (time, y, x) int16 41MB dask.array<chunksize=(1, 3080, 6604), meta=np.ndarray>
    bd_sl3       (time, y, x) int16 41MB dask.array<chunksize=(1, 3080, 6604), meta=np.ndarray>
    bd_sl4       (time, y, x) int16 41MB dask.array<chunksize=(1, 3080, 6604), meta=np.ndarray>
    bd_sl5       (time, y, x) int16 41MB dask.array<chunksize=(1, 3080, 6604), meta=np.ndarray>
    bd_sl6       (time, y, x) int16 41MB dask.array<chunksize=(1, 3080, 6604), meta=np.ndarray>

In [9]:
sys.path.append("/home/rbalogun/raster-to-stac/")

from raster2stac import Raster2STAC
import xarray as xr
import os

rs2stac = Raster2STAC(
    data = bulk_density,
    title = "SoliGrids Bulk Density data for the European Alps",
    description = "SoilGrids is a system for global digital soil mapping that uses state-of-the-art machine learning methods to map the spatial distribution of soil properties across the globe. SoilGrids prediction models are fitted using over 230 000 soil profile observations from the WoSIS database and a series of environmental covariates. Covariates were selected from a pool of over 400 environmental layers from Earth observation derived products and other environmental information including climate, land cover and terrain morphology. The outputs of SoilGrids are global soil property maps at six standard depth intervals (according to the GlobalSoilMap IUSS working group and its specifications) at a spatial resolution of 250 meters.",
    keywords = ["soil", "buik density", "soilgrids", "WoSIS"],
    providers= [
        {
            "url": "https://www.isric.org/about/vision-mission",
            "name": "ISRIC World Soil Information",
            "roles": [
                "producer"
                ]
        },
        {
            "url": "http://www.eurac.edu",
            "name": "Eurac Research - Institute for Earth Observation",
            "roles": [
                "host"
                ]
        }
    ],
    license="CC-BY-4.0",
    sci_doi = "10.5194/soil-7-217-2021",
    sci_citation = "Poggio, L., de Sousa, L. M., Batjes, N. H., Heuvelink, G. B. M., Kempen, B., Ribeiro, E., and Rossiter, D.: SoilGrids 2.0: producing soil information for the globe with quantified spatial uncertainty, SOIL, 7, 217–240, https://doi.org/10.5194/soil-7-217-2021, 2021.",
    collection_id = "BD_SOILGRIDS", # The Collection id we want to set
    collection_url = "http://10.8.244.74:8082/collections/", # The URL where the collection will be exposed
    output_folder = "./stac/BD_SOILGRIDS/",
    s3_upload = False
).generate_cog_stac()